In [26]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import psycopg2
import logging
import pandas as pd
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import FloatType

# Initialiser SparkSession
spark = SparkSession.builder \
    .appName("AnalyseDeSentiment") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.2.25") \
    .getOrCreate()

# Paramètres de connexion PostgreSQL
db_url = "jdbc:postgresql://projetfinal5spar-postgres-1:5432/mastodon_data"
db_properties = {
    "user": "user",
    "password": "password",
    "driver": "org.postgresql.Driver"
}

# Charger les données historiques de PostgreSQL
df = spark.read \
    .jdbc(url=db_url, table="mastodon", properties=db_properties)


# Initialiser l'analyseur VADER
analyzer = SentimentIntensityAnalyzer()

# Fonction pour obtenir le score de sentiment
def get_score(text):
    return analyzer.polarity_scores(text)['compound']

# UDF pour appliquer la fonction de sentiment
sentiment = udf(get_score, FloatType())

# Appliquer l'UDF pour obtenir les scores de sentiment
df = df.withColumn("score", sentiment(col("content")))

# Création du champs label et ajout des valeurs
df = df.withColumn("label", when(col("score") >= 0, 1).otherwise(0))



# Afficher le schéma des données
df.printSchema()


print(f"Tokenizer pour diviser le content du tweet en mots")
tokenizer = Tokenizer(inputCol="content", outputCol="words")
tokenized_df = tokenizer.transform(df)

print(f"Convertir les mots en vecteurs de caractéristiques en utilisant CountVectorizer")
vectorizer = CountVectorizer(inputCol="words", outputCol="features")
vectorized_df = vectorizer.fit(tokenized_df).transform(tokenized_df)


# Configuration de la régression logistique
lr = LogisticRegression(featuresCol="features", labelCol="label")

# Création d'un pipeline
pipeline = Pipeline(stages=[tokenizer, vectorizer, lr])

# Diviser les données en ensembles d’entraînement et de test
(train_data, test_data) = df.randomSplit([0.8, 0.2], seed=1234)

# Entraîner le modèle
model = pipeline.fit(train_data)

# Prédire sur les données de test
predictions = model.transform(test_data)

# Évaluer la précision du modèle
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy:.2f}")

# Prédire les sentiments
sentiment_predictions = model.transform(df)


# Paramètres de connexion à PostgreSQL
db_host = "projetfinal5spar-postgres-1"
db_port = "5432"
db_name = "mastodon_data"
db_user = "user"
db_password = "password"
db_url = f"jdbc:postgresql://{db_host}:{db_port}/{db_name}"
db_properties = {
    "user": db_user,
    "password": db_password,
    "driver": "org.postgresql.Driver"
}

# Fonction pour initialiser la table sentiment_results dans PostgreSQL
def initialize_sentiment_results_table():
    print("Initialisation de la table sentiment_results dans PostgreSQL...")
    try:
        conn = psycopg2.connect(
            dbname=db_name,
            user=db_properties["user"],
            password=db_properties["password"],
            host=db_host,
            port=db_port
        )
        conn.autocommit = True
        cursor = conn.cursor()

        create_table_sql = """
        CREATE TABLE IF NOT EXISTS sentiment_results (
            "id" VARCHAR(255),
            content TEXT,
            sentiment VARCHAR(255),
            PRIMARY KEY ("id")
        );
        """

        cursor.execute(create_table_sql)
        print("Table sentiment_results initialisée avec succès dans PostgreSQL.")

        # Ajout d'une contrainte d'unicité si nécessaire
        cursor.execute("""
        ALTER TABLE sentiment_results ADD CONSTRAINT unique_sentiment UNIQUE ("id");
        """)

        cursor.close()
        conn.close()

    except Exception as e:
        logging.error(f"Erreur lors de l'initialisation de la table sentiment_results dans PostgreSQL: {e}")
        print(f"Erreur lors de l'initialisation de la table sentiment_results: {e}")


# Initialiser la table sentiment_results
initialize_sentiment_results_table()


# Fonction pour stocker les résultats dans PostgreSQL
def store_sentiment_results(dataframe):
    print("Stockage des résultats dans PostgreSQL...")
    try:
        # Convertir le DataFrame Spark en DataFrame Pandas
        pandas_df = dataframe.select("id", "content", "prediction").toPandas()

        # Connexion à PostgreSQL
        conn = psycopg2.connect(
            dbname=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port
        )
        conn.autocommit = True
        cursor = conn.cursor()

        # Insertion des résultats
        for _, row in pandas_df.iterrows():
            cursor.execute("""
                INSERT INTO sentiment_results (id, content, sentiment)
                VALUES (%s, %s, %s)
                ON CONFLICT (id) DO UPDATE SET
                content = EXCLUDED.content,
                sentiment = EXCLUDED.sentiment
            """, (
                row['id'],
                row['content'],
                row['prediction']
            ))

        print("Résultats stockés avec succès dans 'sentiment_results'.")
        cursor.close()
        conn.close()

    except Exception as e:
        logging.error(f"Erreur lors de l'insertion dans 'sentiment_results' : {e}")
        print(f"Erreur lors de l'insertion dans 'sentiment_results' : {e}")

# Appeler la fonction pour stocker les résultats
store_sentiment_results(sentiment_predictions)


print("Traitement terminé.")


root
 |-- id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user: string (nullable = true)
 |-- followers_count: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- content: string (nullable = true)
 |-- language: string (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- favourites_count: integer (nullable = true)
 |-- reblogs_count: integer (nullable = true)
 |-- score: float (nullable = true)
 |-- label: integer (nullable = false)

Tokenizer pour diviser le content du tweet en mots
Convertir les mots en vecteurs de caractéristiques en utilisant CountVectorizer
Accuracy: 1.00


ERROR:root:Erreur lors de l'initialisation de la table sentiment_results dans PostgreSQL: relation "unique_sentiment" already exists



Initialisation de la table mastodon dans PostgreSQL...
Table sentiment_results initialisée avec succès dans PostgreSQL.
Erreur lors de l'initialisation de la table sentiment_results: relation "unique_sentiment" already exists

Stockage des résultats dans PostgreSQL...
Résultats stockés avec succès dans 'sentiment_results'.
Traitement terminé.
